In [1]:
import numpy as np
import pandas as pd
import tensorflow_decision_forests as tfdf
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import tensorflow as tf

ModuleNotFoundError: No module named 'tensorflow_decision_forests'

In [ ]:
titanic_path = 'train.csv'
train = pd.read_csv(titanic_path)
test = pd.read_csv('test.csv')



def clean(df):
    titanic_df=df.copy()

    titanic_df = titanic_df.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)

    titanic_df['Age'].fillna(titanic_df['Age'].median(), inplace=True)
    titanic_df['Embarked'].fillna(titanic_df['Embarked'].mode()[0], inplace=True)
    titanic_df['Fare'].fillna(titanic_df['Fare'].median(), inplace=True)

    titanic_df = pd.get_dummies(titanic_df, columns=['Sex', 'Embarked'], drop_first=True).astype(int)
    


    return titanic_df


train=clean(train)
train_x=train.drop('Survived',axis=1)
train_y=train['Survived']
test=clean(test)


# le=preprocessing.LabelEncoder()

# for col in ['Sex', 'Embarked']:
#     train_x[col]=le.fit_transform(train_x[col])
#     test[col]=le.transform(test[col])
#     print(le.classes_)
    


# x_train,x_val,y_train,y_val=train_test_split(train_x,train_y,test_size=0.2,random_state=19)

# test.shape


In [ ]:
train_x.to_dict('list')

{'Pclass': [3,
  1,
  3,
  1,
  3,
  3,
  1,
  3,
  3,
  2,
  3,
  1,
  3,
  3,
  3,
  2,
  3,
  2,
  3,
  3,
  2,
  2,
  3,
  1,
  3,
  3,
  3,
  1,
  3,
  3,
  1,
  1,
  3,
  2,
  1,
  1,
  3,
  3,
  3,
  3,
  3,
  2,
  3,
  2,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  1,
  2,
  1,
  1,
  2,
  3,
  2,
  3,
  3,
  1,
  1,
  3,
  1,
  3,
  2,
  3,
  3,
  3,
  2,
  3,
  2,
  3,
  3,
  3,
  3,
  3,
  2,
  3,
  3,
  3,
  3,
  1,
  2,
  3,
  3,
  3,
  1,
  3,
  3,
  3,
  1,
  3,
  3,
  3,
  1,
  1,
  2,
  2,
  3,
  3,
  1,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  1,
  3,
  3,
  3,
  3,
  3,
  3,
  2,
  1,
  3,
  2,
  3,
  2,
  2,
  1,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  2,
  2,
  2,
  1,
  1,
  3,
  1,
  3,
  3,
  3,
  3,
  2,
  2,
  3,
  3,
  2,
  2,
  2,
  1,
  3,
  3,
  3,
  1,
  3,
  3,
  3,
  3,
  3,
  2,
  3,
  3,
  3,
  3,
  1,
  3,
  1,
  3,
  1,
  3,
  3,
  3,
  1,
  3,
  3,
  1,
  2,
  3,
  3,
  2,
  3,
  2,
  3,
  1,
  3,
  1,
  3,
  3,
  2,
  2,
  3,
  2,
  1,
  1,
  3,
  3,


In [ ]:
# train['Embarked']=pd.Categorical(train['Embarked'])
# train['Embarked']=train.Embarked.cat.codes
dataset=tf.data.Dataset.from_tensor_slices((train_x.to_dict('list'),train_y.values))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import tensorflow as tf

# model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
# model.fit(x_train,y_train)
# accuracy_score(y_val,predictions)

# model=LogisticRegression(random_state=19,max_iter=1000)
# model.fit(x_train,y_train)
# predictions = model.predict(x_val)

# model=tf.keras.Sequential([
#     tf.keras.layers.Input(shape=(x_train.shape[1],)),
#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dense(1,activation='sigmoid')
# ])
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# history=model.fit(x_train,y_train, epochs=100, batch_size=32, validation_data=(x_val,y_val))

# 


# predictions = model.predict(test)
# accuracy_score(y_val.values,model.predict(x_val).flatten())

tf_train_dataset = tfdf.keras.pd_dataframe_to_tf_dataset(train, label='Survived')
tf_test = tfdf.keras.pd_dataframe_to_tf_dataset(test)
model = tfdf.keras.GradientBoostedTreesModel(
    verbose=0, # Very few logs
    # features=[tfdf.keras.FeatureUsage(name=n) for n in input_features],
    # exclude_non_specified_features=True, # Only use the features in "features"
    random_seed=1234,
)
model.fit(tf_train_dataset)
prediction = model.predict(tf_test)
predictions=(prediction>=0.5).astype(int)
predictions=predictions.reshape(1,-1)
predictions=predictions[0]

[WARNING 23-11-26 18:22:53.4914 PST gradient_boosted_trees.cc:1886] "goss_alpha" set but "sampling_method" not equal to "GOSS".
[WARNING 23-11-26 18:22:53.4916 PST gradient_boosted_trees.cc:1897] "goss_beta" set but "sampling_method" not equal to "GOSS".
[WARNING 23-11-26 18:22:53.4916 PST gradient_boosted_trees.cc:1911] "selective_gradient_boosting_ratio" set but "sampling_method" not equal to "SELGB".
[INFO 23-11-26 18:22:55.0988 PST kernel.cc:1233] Loading model from path /tmp/tmpz2chginn/model/ with prefix a5130bebd61b4d42
[INFO 23-11-26 18:22:55.0998 PST quick_scorer_extended.cc:903] The binary was compiled without AVX2 support, but your CPU supports it. Enable it for faster model inference.
[INFO 23-11-26 18:22:55.0998 PST abstract_model.cc:1344] Engine "GradientBoostedTreesQuickScorerExtended" built
[INFO 23-11-26 18:22:55.0999 PST kernel.cc:1061] Use fast generic engine


1/1 [==============================] - 0s 20ms/step


In [ ]:
y=model.predict(x_val).flatten()
y=np.where(y >=0.5, 1, 0)
accuracy_score(y_val.values,y)

6/6 [==============================] - 0s 443us/step


0.8156424581005587

predictions = model.predict(test)

In [ ]:
# predictions = model.predict(test)
# predictions=np.where(predictions >=0.5, 1, 0)
pred=predictions.flatten()
pred.shape



(418,)

In [ ]:
te = pd.read_csv('test.csv')
output = pd.DataFrame({'PassengerId': te.PassengerId, 'Survived': pred})
output.to_csv('submission.csv', index=False)
# predictions

In [ ]:
#another approach using tensorflow dataset

titanic=pd.read_csv('train.csv')
titanic_features = titanic.copy()
titanic_labels = titanic_features.pop('Survived')


In [ ]:
# Create a symbolic input
input = tf.keras.Input(shape=(), dtype=tf.float32)

# Perform a calculation using the input
result = 2*input + 1

# the result doesn't have a value
calc = tf.keras.Model(inputs=input, outputs=result)

print(calc(1).numpy())
print(calc(2).numpy())

3.0
5.0


In [ ]:
inputs = {}

for name, column in titanic_features.items():
  dtype = column.dtype

  if dtype == object:
    dtype = tf.string
  else:
    dtype = tf.float32

  inputs[name] = tf.keras.Input(shape=(1,), name=name, dtype=dtype)

inputs

{'PassengerId': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'PassengerId')>,
 'Pclass': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'Pclass')>,
 'Name': <KerasTensor: shape=(None, 1) dtype=string (created by layer 'Name')>,
 'Sex': <KerasTensor: shape=(None, 1) dtype=string (created by layer 'Sex')>,
 'Age': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'Age')>,
 'SibSp': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'SibSp')>,
 'Parch': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'Parch')>,
 'Ticket': <KerasTensor: shape=(None, 1) dtype=string (created by layer 'Ticket')>,
 'Fare': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'Fare')>,
 'Cabin': <KerasTensor: shape=(None, 1) dtype=string (created by layer 'Cabin')>,
 'Embarked': <KerasTensor: shape=(None, 1) dtype=string (created by layer 'Embarked')>}

In [ ]:
from tensorflow.keras import layers

numeric_inputs = {name:input for name,input in inputs.items()
                  if input.dtype==tf.float32}

x = layers.Concatenate()(list(numeric_inputs.values()))

norm = layers.Normalization()
norm.adapt(np.array(titanic[numeric_inputs.keys()]))
all_numeric_inputs = norm(x)

all_numeric_inputs

<KerasTensor: shape=(None, 6) dtype=float32 (created by layer 'normalization_2')>

In [ ]:
preprocessed_inputs = [all_numeric_inputs]

In [ ]:
for name, input in inputs.items():
  if input.dtype == tf.float32:
    continue
  
  lookup = layers.StringLookup(vocabulary=np.unique(titanic_features[name]))
  one_hot = layers.CategoryEncoding(num_tokens=lookup.vocabulary_size())

  x = lookup(input)
  x = one_hot(x)
  preprocessed_inputs.append(x)

TypeError: '<' not supported between instances of 'str' and 'float'

In [ ]:
# pipeline
titanic_csv_ds = tf.data.experimental.make_csv_dataset(
    'train.csv',
    batch_size=5, # Artificially small to make examples easier to show.
    label_name='Survived',
    num_epochs=1,
    ignore_errors=True,)

Instructions for updating:
Use `tf.data.Dataset.ignore_errors` instead.


In [ ]:
for batch, label in titanic_csv_ds.take(1):
  for key, value in batch.items():
    print(f"{key:20s}: {value}")
  print()
  print(f"{'label':20s}: {label}")

PassengerId         : [261  80 202 515 808]
Pclass              : [3 3 3 3 3]
Name                : [b'Smith, Mr. Thomas' b'Dowdell, Miss. Elizabeth' b'Sage, Mr. Frederick'
 b'Coleff, Mr. Satio' b'Pettersson, Miss. Ellen Natalia']
Sex                 : [b'male' b'female' b'male' b'male' b'female']
Age                 : [ 0. 30.  0. 24. 18.]
SibSp               : [0 0 8 0 0]
Parch               : [0 0 2 0 0]
Ticket              : [b'384461' b'364516' b'CA. 2343' b'349209' b'347087']
Fare                : [ 7.75   12.475  69.55    7.4958  7.775 ]
Cabin               : [b'' b'' b'' b'' b'']
Embarked            : [b'Q' b'S' b'S' b'S' b'S']

label               : [0 1 0 0 0]


In [ ]:
tf.data.tex